https://github.com/hananedupouy/LLMs-in-Finance/blob/main/Multimodal_llms/financial_analysis/multimodal_claude3_5_sonnet_gpt4o_claude3_opus_charts_analysis.ipynb

https://github.com/run-llama/llama_parse/blob/main/examples/demo_json.ipynb

https://docs.llamaindex.ai/en/stable/module_guides/models/multi_modal/

https://stackoverflow.com/questions/77727695/google-gemini-api-error-defaultcredentialserror-your-default-credentials-were


In [ ]:
import nest_asyncio
nest_asyncio.apply()

import os
from PIL import Image

import google.generativeai as genai

from llama_index.core import SimpleDirectoryReader
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index.multi_modal_llms.anthropic import AnthropicMultiModal
from llama_index.multi_modal_llms.gemini import GeminiMultiModal
# genai.configure(api_key=os.environ.get('GAI_API_KEY'))
# genai.configure(api_key=os.environ['GAI_API_KEY'])

/Users/gurdeep/venvs/pdfplumber/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/gurdeep/venvs/pdfplumber/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# genai.configure(api_key=os.environ.get('GAI_API_KEY'))
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
max_tokens = 1500

In [3]:
# openai_mm_llm = OpenAIMultiModal(model="gpt-4-vision-preview", max_new_tokens=500)
gpt4o = OpenAIMultiModal(model="gpt-4o", max_new_tokens=max_tokens)
sonnet_35 = AnthropicMultiModal(model="claude-3-5-sonnet-20240620", max_new_tokens=max_tokens)
gemini_15_pro = GeminiMultiModal(model_name="models/gemini-1.5-pro", max_new_tokens=max_tokens)

I0000 00:00:1722081374.750984    4804 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1722081374.813907    4804 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [4]:
# openai_mm_llm = OpenAIMultiModal(model="gpt-4-vision-preview", max_new_tokens=500)
gpt4o_mini = OpenAIMultiModal(model="gpt-4o-mini", max_new_tokens=max_tokens)
haiku3 = AnthropicMultiModal(model="claude-3-haiku-20240307", max_new_tokens=max_tokens)
gemini_15_flash = GeminiMultiModal(model_name="models/gemini-1.5-flash", max_new_tokens=max_tokens)

I0000 00:00:1722081376.961062    4804 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [5]:
# imgs = {}
# img_path = '../data/esi/test'
# for item in os.listdir(img_path):
#     # print(item)
#     img = Image.open(os.path.join(img_path, item))
#     # img.show()
#     imgs[item] = img

In [6]:
# for img in imgs.keys():
#     print(img)
#     image_file = imgs[img]
#     # img = Image.open(image_file)
#     image_file.show()

In [5]:
os.getcwd()

'/Users/gurdeep/GitHub/fine_tune_llm_course/notebooks'

In [7]:
image_list = []
img_path = '../data/esi/test'

for item in os.listdir(img_path):
    # print(item)
    if item.endswith('.png') or item.endswith('.jpg'):
        # check image
        # image_file = Image.open(os.path.join(img_path, item))
        # image_file.show()
        # image_list.append(image_file)
        image_list.append(os.path.join(img_path,item))

print(image_list)

['../data/esi/test/pg_51_img_0.png', '../data/esi/test/pg_51_img_1.png', '../data/esi/test/pg_51_img_3.png', '../data/esi/test/pg_51_img_2.png', '../data/esi/test/pg_49_img_2.png', '../data/esi/test/pg_49_img_0.png', '../data/esi/test/pg_49_img_1.png']


In [8]:
prompt="""
    you are an analyst reading tables and charts with the objective of extracting information from them for analysis.
    Identify the image as a table or a chart. 
    If the image is a table then return the table in markdown format.
    If the image is a chart then extract the information in the chart from the image in as much detail as possible. Do not draw any interpretaions but focus solely on extracting information from the image
    If you are unable to classify the image as either a table or a chart, please summarize the image and return the summary.
"""

In [10]:
image_documents = SimpleDirectoryReader(input_files=[image_list[3]]).load_data()

In [11]:
# for img in image_list:
#     img.show()

In [12]:
gpt4o_mini_response = gpt4o_mini.complete(prompt=prompt, image_documents=image_documents)
print(gpt4o_mini_response)

I0000 00:00:1722059395.359956   53085 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork


The image is a chart. 

### Summary of the Chart:
- **Title**: Year-over-Year (YoY) Percentage Change in CPI and CPI-Food
- **Axes**:
  - **X-axis**: Time (from February 2021 to December 2022)
  - **Y-axis**: Percentage (%)
- **Lines**:
  - **CPI (Consumer Price Index)**: Represented by a solid blue line, showing fluctuations over the specified period.
  - **CPI-Food**: Represented by a dashed orange line, also showing fluctuations but with a different trend compared to the CPI line.
- **Trends**:
  - Both lines exhibit variations over time, with CPI generally showing a more stable trend compared to the more volatile CPI-Food line.
  - Notable peaks and troughs can be observed, indicating periods of inflation or deflation in consumer prices and food prices.

This chart provides insights into the relationship between overall consumer prices and food prices over the specified timeframe.


In [13]:
gpt4o_response = gpt4o.complete(prompt=prompt, image_documents=image_documents)
print(gpt4o_response)

The image is a chart. It appears to be a line chart comparing the year-over-year (YoY) percentage changes in the Consumer Price Index (CPI) and CPI-Food over a period from February 2021 to December 2022.

Here is a detailed extraction of the information from the chart:

- **X-Axis (Time Period):** The time period spans from February 2021 to December 2022, with data points marked at intervals of approximately two months.
- **Y-Axis (Percentage Change):** The Y-axis represents the percentage change (YoY) in CPI and CPI-Food, ranging from 0% to 8%.

- **CPI (Blue Line):**
  - February 2021: Approximately 5%
  - April 2021: Approximately 4%
  - June 2021: Approximately 6%
  - August 2021: Approximately 5%
  - October 2021: Approximately 4%
  - December 2021: Approximately 5%
  - February 2022: Approximately 6%
  - April 2022: Approximately 7%
  - June 2022: Approximately 8%
  - August 2022: Approximately 7%
  - October 2022: Approximately 6%
  - December 2022: Approximately 5%

- **CPI-Foo

In [14]:
len(gpt4o_response.text), len(gpt4o_mini_response.text)

(1847, 897)

In [9]:
t1_llms = [gpt4o, sonnet_35, gemini_15_pro]
t2_llms = [gpt4o_mini, haiku3, gemini_15_flash]

In [10]:
%%capture output
image_descr_t1 = {}

for img in image_list:
    # print(img)
    image_descr_t1[img] = []

for image_file in image_descr_t1.keys():
    for llm in t1_llms:
        image_documents = SimpleDirectoryReader(input_files=[image_file]).load_data()
        response = llm.complete(prompt=prompt, image_documents=image_documents)
        # print(restponse.text)
        image_descr_t1[image_file].append(response.text)
        print(f'{llm} done!')
        # break
    

I0000 00:00:1722081406.786402    4804 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1722081411.209308    4804 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1722081418.060534    4804 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [17]:
image_descr_t1

{'../data/esi/test/pg_51_img_0.png': ['The image is a chart. Here is the detailed information extracted from the chart:\n\n- The chart shows the percentage change over different fiscal years (FY) for three sectors: Agriculture and allied activities, Industry, and Services.\n- The fiscal years represented are FY19 (3rd RE), FY20 (2nd RE), FY21 (1st RE), FY22 (PE), and FY23 (1st AE).\n- The y-axis represents the percentage change, ranging from -12% to 9%.\n- The three sectors are represented by different line styles:\n  - Agriculture and allied activities: Solid green line\n  - Industry: Dashed orange line\n  - Services: Dotted blue line\n\nPercentage changes for each sector over the fiscal years:\n- **Agriculture and allied activities:**\n  - FY19 (3rd RE): Approximately 2.5%\n  - FY20 (2nd RE): Approximately 4%\n  - FY21 (1st RE): Approximately 3.5%\n  - FY22 (PE): Approximately 3%\n  - FY23 (1st AE): Approximately 3.5%\n\n- **Industry:**\n  - FY19 (3rd RE): Approximately 6%\n  - FY20 

In [13]:
import csv

In [21]:
filename = '../data/esi/test/image_descr.csv'
headers = ['filename','description']
# open the file for writing
with open(filename, mode='w', newline='')as file:
    writer = csv.writer(file)
    
    # write the header - dictionary keys
    writer.writerow(headers)
    
    # write rows - dictionary values
    for key, value in image_descr_t1.items():
        writer.writerow([key, value[0]])

In [17]:
%%capture output
image_descr_t2 = {}

for img in image_list:
    # print(img)
    image_descr_t2[img] = []

for image_file in image_descr_t2.keys():
    for llm in t2_llms:
        image_documents = SimpleDirectoryReader(input_files=[image_file]).load_data()
        response = llm.complete(prompt=prompt, image_documents=image_documents)
        # print(restponse.text)
        image_descr_t2[image_file].append(response.text)
        print(f'{llm} done!')
        # break
    

In [18]:
# image_descr2 = {}
# llms = [openai_mm_llm, anthropic_mm_llm, gemini_mm_llm]

# for llm in llms:
#     image_descr2[llm] = []

# for image_file in image_descr2.keys():
#     for img in image_list:
#         image_documents = SimpleDirectoryReader(input_files=[img]).load_data()
#         response = llm.complete(prompt=prompt, image_documents=image_documents)
#         print(restponse.text)
#         image_descr2[llm].append(response.text)
#         print(f'{llm} done!')
#         break


In [19]:
import pandas as pd
pd.set_option('display.width',500)
pd.set_option('display.notebook_repr_html',True)
pd.set_option('display.max_colwidth',500)

from IPython.display import HTML

In [42]:
image_df_t1 = pd.DataFrame(image_descr_t1).transpose().reset_index()
# print(image_df_t1.head())
image_df_t1.columns = ['Image', 'gpt4o', 'sonnet_35', 'gemini_15_pro']
# image_df.columns = ['OpenAI', 'Anthropic','Gemini']
print(image_df_t1.shape)


(7, 4)


In [32]:
image_df_t1.head(1)

,Image,gpt4o,sonnet_35,gemini_15_pro
0,../data/esi/test/pg_51_img_0.png,"The image is a chart.\n\n### Extracted Information:\n\n#### Title: \n- The chart does not have a visible title.\n\n#### Axes:\n- **Y-Axis (Vertical):** Represents ""Per cent"" with values ranging from -12 to 8.\n- **X-Axis (Horizontal):** Represents fiscal years (FY) with the following labels:\n - FY19 (3rd RE)\n - FY20 (2nd RE)\n - FY21 (1st RE)\n - FY22 (PE)\n - FY23 (1st AE)\n\n#### Legend:\n- **Green Solid Line:** Agriculture and allied activities\n- **Orange Dashed Line:** Industry\n...","This image is a chart.\n\nThe chart displays the percentage change for three economic sectors over several fiscal years:\n\n1. Agriculture and allied activities (solid green line)\n2. Industry (dashed orange line)\n3. Services (dashed blue line)\n\nThe x-axis shows fiscal years from FY19 (3rd RE) to FY23 (1st AE), with FY22 (PE) in between.\nThe y-axis represents the percentage change, ranging from -12% to 8%.\n\nKey observations:\n\n1. Agriculture and allied activities show the most stable ...","The image is a chart.\n\nThe chart shows the percentage change in Gross Value Added (GVA) for three sectors - Agriculture and allied activities, Industry, and Services - over five fiscal years: FY19 (3rd RE), FY20 (2nd RE), FY21 (1st RE), FY22 (PE), and FY23 (1st AE).\n\nHere's a summary of the trends:\n\n* **Agriculture and allied activities (Green):** This sector shows a steady increase in GVA from FY19 to FY21, peaking at around 7%. It then stabilizes around 3% for FY22 and FY23.\n* **Ind..."


In [43]:
def path_to_image_html(path):
    return f'<img src="{path}" width="200" >'

In [44]:
image_df_t1['Image'] = image_df_t1['Image'].apply(path_to_image_html)
image_df_t1.head(1)

,Image,gpt4o,sonnet_35,gemini_15_pro
0,"<img src=""../data/esi/test/pg_51_img_0.png"" width=""200"" >","The image is a chart.\n\n### Extracted Information:\n\n- **Y-Axis:** Represents ""Per cent"" with values ranging from -12 to 9.\n- **X-Axis:** Represents fiscal years:\n - FY19 (3rd RE)\n - FY20 (2nd RE)\n - FY21 (1st RE)\n - FY22 (PE)\n - FY23 (1st AE)\n\n- **Legend:**\n - Green solid line: Agriculture and allied activities\n - Orange dashed line: Industry\n - Blue dashed line: Services\n\n### Data Points:\n\n- **Agriculture and allied activities (Green solid line):**\n - FY19: Appro...","This image is a chart.\n\nThe chart displays the percentage change for three economic sectors over several fiscal years:\n\n1. Agriculture and allied activities (solid green line)\n2. Industry (dashed orange line)\n3. Services (dashed blue line)\n\nThe x-axis shows fiscal years:\n- FY19 (3rd RE)\n- FY20 (2nd RE)\n- FY21 (1st RE)\n- FY22 (PE)\n- FY23 (1st AE)\n\nThe y-axis represents the percentage change, ranging from -12% to 8%.\n\nKey observations from the chart:\n\n1. Agriculture and alli...","The image is a chart. \n\nThe chart plots three lines across five different points in time. \n\n* **Vertical axis:** Represents ""Per cent"" ranging from -12 to 8.\n* **Horizontal axis:** Shows five different fiscal years with their respective quarters:\n * FY19 (3rd RE)\n * FY20 (2nd RE)\n * FY21 (1st RE)\n * FY22 (PE)\n * FY23 (1st AE)\n\n* **Lines:** Three lines are present, each representing a sector:\n * Green line: Agriculture and allied activities\n * Orange dashed ..."


In [45]:
HTML(image_df_t1.to_html(escape=False))

,Image,gpt4o,sonnet_35,gemini_15_pro
0,,"The image is a chart.\n\n### Extracted Information:\n\n- **Y-Axis:** Represents ""Per cent"" with values ranging from -12 to 9.\n- **X-Axis:** Represents fiscal years:\n - FY19 (3rd RE)\n - FY20 (2nd RE)\n - FY21 (1st RE)\n - FY22 (PE)\n - FY23 (1st AE)\n\n- **Legend:**\n - Green solid line: Agriculture and allied activities\n - Orange dashed line: Industry\n - Blue dashed line: Services\n\n### Data Points:\n\n- **Agriculture and allied activities (Green solid line):**\n - FY19: Approximately 2.5%\n - FY20: Approximately 4%\n - FY21: Approximately 3.5%\n - FY22: Approximately 3%\n - FY23: Approximately 3.5%\n\n- **Industry (Orange dashed line):**\n - FY19: Approximately 6.5%\n - FY20: Approximately -1.5%\n - FY21: Approximately -7%\n - FY22: Approximately 11%\n - FY23: Approximately 4%\n\n- **Services (Blue dashed line):**\n - FY19: Approximately 7.5%\n - FY20: Approximately 5.5%\n - FY21: Approximately -8%\n - FY22: Approximately 8.5%\n - FY23: Approximately 6.5%","This image is a chart.\n\nThe chart displays the percentage change for three economic sectors over several fiscal years:\n\n1. Agriculture and allied activities (solid green line)\n2. Industry (dashed orange line)\n3. Services (dashed blue line)\n\nThe x-axis shows fiscal years:\n- FY19 (3rd RE)\n- FY20 (2nd RE)\n- FY21 (1st RE)\n- FY22 (PE)\n- FY23 (1st AE)\n\nThe y-axis represents the percentage change, ranging from -12% to 8%.\n\nKey observations from the chart:\n\n1. Agriculture and allied activities:\n - Starts around 2% in FY19\n - Peaks at about 5% in FY20\n - Remains relatively stable around 3% from FY21 to FY23\n\n2. Industry:\n - Starts at about 5% in FY19\n - Declines to approximately -2% in FY21\n - Sharply rises to about 9% in FY22\n - Drops to around 4% in FY23\n\n3. Services:\n - Begins at about 7% in FY19\n - Sharply declines to about -8% in FY21\n","The image is a chart. \n\nThe chart plots three lines across five different points in time. \n\n* **Vertical axis:** Represents ""Per cent"" ranging from -12 to 8.\n* **Horizontal axis:** Shows five different fiscal years with their respective quarters:\n * FY19 (3rd RE)\n * FY20 (2nd RE)\n * FY21 (1st RE)\n * FY22 (PE)\n * FY23 (1st AE)\n\n* **Lines:** Three lines are present, each representing a sector:\n * Green line: Agriculture and allied activities\n * Orange dashed line: Industry\n * Blue dashed line: Services \n"
1,,The image is a chart. Here is the detailed information extracted from the chart:\n\n### Categories:\n1. PFCE (Private Final Consumption Expenditure)\n2. GFCF (Gross Fixed Capital Formation)\n3. Exports of goods and services\n4. Imports of goods and services\n\n### Fiscal Years and Corresponding Colors:\n- FY19 (3rd RE): Yellow\n- FY20 (2nd RE): Orange\n- FY21 (1st RE): Purple\n- FY22 (PE): Green\n- FY23 (1st AE): Blue\n\n### Data Points (Per cent of GDP):\n\n#### PFCE:\n- FY19: ~59%\n- FY20: ~58%\n- FY21: ~60%\n- FY22: ~60%\n- FY23: ~60%\n\n#### GFCF:\n- FY19: ~28%\n- FY20: ~26%\n- FY21: ~27%\n- FY22: ~29%\n- FY23: ~30%\n\n#### Exports of goods and services:\n- FY19: ~20%\n- FY20: ~18%\n- FY21: ~20%\n- FY22: ~22%\n- FY23: ~23%\n\n#### Imports of goods and services:\n- FY19: ~23%\n- FY20: ~20%\n- FY21: ~21%\n- FY22: ~23%\n- FY23: ~24%,"This image is a chart, specifically a grouped bar chart. Here's the detailed information extracted from the chart:\n\n1. The chart compares five fiscal years: FY19 (3rd RE), FY20 (2nd RE), FY21 (1st RE), FY22 (PE), and FY23 (1st AE).\n\n2. The vertical axis represents ""Per cent of GDP"" and ranges from 0 to 80.\n\n3. The horizontal axis shows four categories: PFCE, GFCF, Exports of goods and services, and Imports of goods and services.\n\n4. Data for PFCE (Private Final Consumption Expenditure):\n - All fiscal years show values around 56-57% of GDP\n - Very little variation across years\n\n5. Data for GFCF (Gross Fixed Capital Formation):\n - Values range between approximately 30-35% of GDP\n - FY23 shows the hi

In [46]:
image_df_t1.columns

Index(['Image', 'gpt4o', 'sonnet_35', 'gemini_15_pro'], dtype='object')

In [47]:
# Define a style function to set minimum width
def style_min_width(s, width):
    return [f'width: {width}px;' for _ in s]

# Apply the style function
styled_df = image_df_t1.style.apply(style_min_width, width=250, subset=['Image'])

# Display the styled DataFrame
display(styled_df)

,Image,gpt4o,sonnet_35,gemini_15_pro
0,,"The image is a chart. ### Extracted Information: - **Y-Axis:** Represents ""Per cent"" with values ranging from -12 to 9. - **X-Axis:** Represents fiscal years: - FY19 (3rd RE) - FY20 (2nd RE) - FY21 (1st RE) - FY22 (PE) - FY23 (1st AE) - **Legend:** - Green solid line: Agriculture and allied activities - Orange dashed line: Industry - Blue dashed line: Services ### Data Points: - **Agriculture and allied activities (Green solid line):** - FY19: Approximately 2.5% - FY20: Approximately 4% - FY21: Approximately 3.5% - FY22: Approximately 3% - FY23: Approximately 3.5% - **Industry (Orange dashed line):** - FY19: Approximately 6.5% - FY20: Approximately -1.5% - FY21: Approximately -7% - FY22: Approximately 11% - FY23: Approximately 4% - **Services (Blue dashed line):** - FY19: Approximately 7.5% - FY20: Approximately 5.5% - FY21: Approximately -8% - FY22: Approximately 8.5% - FY23: Approximately 6.5%","This image is a chart. The chart displays the percentage change for three economic sectors over several fiscal years: 1. Agriculture and allied activities (solid green line) 2. Industry (dashed orange line) 3. Services (dashed blue line) The x-axis shows fiscal years: - FY19 (3rd RE) - FY20 (2nd RE) - FY21 (1st RE) - FY22 (PE) - FY23 (1st AE) The y-axis represents the percentage change, ranging from -12% to 8%. Key observations from the chart: 1. Agriculture and allied activities: - Starts around 2% in FY19 - Peaks at about 5% in FY20 - Remains relatively stable around 3% from FY21 to FY23 2. Industry: - Starts at about 5% in FY19 - Declines to approximately -2% in FY21 - Sharply rises to about 9% in FY22 - Drops to around 4% in FY23 3. Services: - Begins at about 7% in FY19 - Sharply declines to about -8% in FY21","The image is a chart. The chart plots three lines across five different points in time. * **Vertical axis:** Represents ""Per cent"" ranging from -12 to 8. * **Horizontal axis:** Shows five different fiscal years with their respective quarters: * FY19 (3rd RE) * FY20 (2nd RE) * FY21 (1st RE) * FY22 (PE) * FY23 (1st AE) * **Lines:** Three lines are present, each representing a sector: * Green line: Agriculture and allied activities * Orange dashed line: Industry * Blue dashed line: Services"
1,,The image is a chart. Here is the detailed information extracted from the chart: ### Categories: 1. PFCE (Private Final Consumption Expenditure) 2. GFCF (Gross Fixed Capital Formation) 3. Exports of goods and services 4. Imports of goods and services ### Fiscal Years and Corresponding Colors: - FY19 (3rd RE): Yellow - FY20 (2nd RE): Orange - FY21 (1st RE): Purple - FY22 (PE): Green - FY23 (1st AE): Blue ### Data Points (Per cent of GDP): #### PFCE: - FY19: ~59% - FY20: ~58% - FY21: ~60% - FY22: ~60% - FY23: ~60% #### GFCF: - FY19: ~28% - FY20: ~26% - FY21: ~27% - FY22: ~29% - FY23: ~30% #### Exports of goods and services: - FY19: ~20% - FY20: ~18% - FY21: ~20% - FY22: ~22% - FY23: ~23% #### Imports of goods and services: - FY19: ~23% - FY20: ~20% - FY21: ~21% - FY22: ~23% - FY23: ~24%,"This image is a chart, specifically a grouped bar chart. Here's the detailed information extracted from the chart: 1. The chart compares five fiscal years: FY19 (3rd RE), FY20 (2nd RE), FY21 (1st RE), FY22 (PE), and FY23 (1st AE). 2. The vertical axis represents ""Per cent of GDP"" and ranges from 0 to 80. 3. The horizontal axis shows four categories: PFCE, GFCF, Exports of goods and services, and Imports of goods and services. 4. Data for PFCE (Private Final Consumption Expenditure): - All fiscal years show values around 56-57% of GDP - Very little variation across years 5. Data for GFCF (Gross Fixed Capital Formation): - Values range between approximately 30-35% of GDP - FY23 shows the highest value, followed by FY22 - FY21 shows the lowest value 6. Data for Exports of goods and services: - Values range between approximately 18-23% of GDP - FY22 and FY23 show higher values compared to previous years - FY20 

In [39]:
image_df_t2 = pd.DataFrame(image_descr_t2).transpose().reset_index()
image_df_t2.columns = ['Image','gpt4o_mini', 'haiku3', 'gemini_15_flash']
# image_df.columns = ['OpenAI', 'Anthropic','Gemini']
print(image_df_t2.shape)

image_df_t2['Image'] = image_df_t2['Image'].apply(path_to_image_html)

# Apply the style function
styled_df = image_df_t2.style.apply(style_min_width, width=250, subset=['Image'])

# Display the styled DataFrame
display(styled_df)

(7, 4)


,Image,gpt4o_mini,haiku3,gemini_15_flash
0,,"The image is a chart. ### Summary of the Chart: - The chart displays three lines representing different sectors: Agriculture and allied activities, Industry, and Services. - The y-axis indicates the percentage change, ranging from -12% to 8%. - The x-axis represents fiscal years: FY19 (3rd RE), FY20 (2nd RE), FY21 (1st RE), FY22 (PE), and FY23 (1st AE). - The green line represents Agriculture and allied activities, showing a positive trend. - The orange dashed line represents Industry, which fluctuates but remains below the green line. - The blue dashed line represents Services, which shows a downward trend in the earlier years but stabilizes in FY22 and FY23. This chart provides insights into the performance trends of these sectors over the specified fiscal years.","The image appears to be a chart or graph, not a table. The chart displays the percent change over time for three different sectors: Agriculture and allied activities, Industry, and Services. The x-axis shows the years from FY19 (3rd RE) to FY23 (1st AE), and the y-axis shows the percent change ranging from -12% to 8%. The green line represents the percent change for Agriculture and allied activities, the orange line represents the percent change for Industry, and the blue line represents the percent change for Services. The chart shows the fluctuations in the percent change for each sector over the given time period, allowing for analysis of the trends and performance of these different economic sectors.","The image is a chart. The chart shows the growth rate of three sectors: Agriculture and allied activities, Industry, and Services, over the period FY19 (3rd RE) to FY23 (1st AE). The chart shows that: * Agriculture and allied activities grew steadily from FY19 to FY21, then remained relatively stable in FY22 and FY23. * Industry experienced a decline in growth from FY19 to FY21, then rebounded in FY22 before declining again in FY23. * Services experienced a decline in growth from FY19 to FY21, then rebounded in FY22 and remained relatively stable in FY23. The chart suggests that the Indian economy is experiencing a period of transition, with the services sector becoming increasingly important. However, the industry sector is also showing signs of recovery, which could lead to further growth in the future."
1,,"The image is a chart. ### Summary of the Chart: The chart displays the percentage of GDP for various economic indicators across different fiscal years (FY19, FY20, FY21, FY22, and FY23). The indicators represented are: - **PFCE (Private Final Consumption Expenditure)** - **GFCF (Gross Fixed Capital Formation)** - **Exports of goods and services** - **Imports of goods and services** Each fiscal year is represented by a different color, allowing for a visual comparison of how these indicators have changed over the years. The y-axis indicates the percentage of GDP, while the x-axis lists the economic indicators. The chart provides insights into trends in consumption, investment, and trade over the specified fiscal years.","The image appears to be a chart that displays various economic indicators over different fiscal years. The chart shows the following information: 1. PFCE (Private Final Consumption Expenditure): This indicator shows a steady increase from FY19 to FY22, with the highest value in FY22. 2. GFCF (Gross Fixed Capital Formation): This indicator also shows an increasing trend from FY19 to FY23, with the highest value in FY23. 3. Exports of goods and services: This indicator fluctuates over the years, with the highest value in FY23 and the lowest in FY21. 4. Imports of goods and services: This indicator also fluctuates, with the highest value in FY23 and the lowest in FY21. The chart provides a visual representation of the trends in these economic indicators across different fiscal years, allowing for a better understanding of the economic performance over time.","The image is a chart. The chart is a bar